# LSTM

In [1]:
import numpy as np
import pandas as pd
import deploy_functions as dpf
import helper_functions as hf

import keras
from keras.models import load_model

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


### Load Data

In [2]:
cow_1 = pd.read_json('input/cowrie.json.2017_7_4', lines=True)
cow_2 = pd.read_json('input/cowrie.json.2017_7_6', lines=True)
cow_3 = pd.read_json('input/cowrie.json.2017_7_7', lines=True)
cow = pd.concat([cow_1, cow_2, cow_3], axis=0)
cow.head()

,compCS,data,dst_ip,dst_port,duration,encCS,eventid,height,input,isError,...,size,src_ip,src_port,system,timestamp,ttylog,url,username,version,width
0,NaN,NaN,10.10.0.13,2222.0,NaN,NaN,cowrie.session.connect,NaN,NaN,0.0,...,NaN,116.31.116.16,39547.0,cowrie.ssh.factory.CowrieSSHFactory,2017-07-03 18:30:07.569246,NaN,NaN,NaN,NaN,NaN
1,[none],NaN,NaN,NaN,NaN,"[aes128-ctr, aes192-ctr, aes256-ctr, aes256-cb...",cowrie.client.version,NaN,NaN,0.0,...,NaN,116.31.116.16,NaN,"HoneyPotSSHTransport,523,116.31.116.16",2017-07-03 18:30:08.235671,NaN,NaN,NaN,SSH-2.0-PUTTY,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,cowrie.login.success,NaN,NaN,0.0,...,NaN,116.31.116.16,NaN,SSHService 'ssh-userauth' on HoneyPotSSHTransp...,2017-07-03 18:30:10.585142,NaN,NaN,root,NaN,NaN
3,NaN,NaN,NaN,NaN,9.96463,NaN,cowrie.session.closed,NaN,NaN,0.0,...,NaN,116.31.116.16,NaN,"HoneyPotSSHTransport,523,116.31.116.16",2017-07-03 18:30:17.902600,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,10.10.0.13,2222.0,NaN,NaN,cowrie.session.connect,NaN,NaN,0.0,...,NaN,116.31.116.16,22040.0,cowrie.ssh.factory.CowrieSSHFactory,2017-07-03 18:30:44.657703,NaN,NaN,NaN,NaN,NaN


### Process data

In [3]:
seq = dpf.proc_data(cow)
print (seq[0])

HBox(children=(IntProgress(value=0, max=22449), HTML(value='')))


[16, 15]


### Load Model

In [4]:
mod = load_model('lstm/lstm_model.h5') 
mod.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 1, 100)            80400     
_________________________________________________________________
lstm_12 (LSTM)               (None, 10)                4440      
Total params: 84,840
Trainable params: 84,840
Non-trainable params: 0
_________________________________________________________________


### Prediction

In [ ]:
d = seq[3]
         
corr = 0
total = 0
pred = []
pred.append(16)

for i in range(1, len(d)):
    temp = d[:i]
    s = [j+1 for j in temp]
    
    if len(s)> 100:
        s = s[len(s)-100:]
    if len(temp) < 100:
        while len(s) is not 100:
            s = [0] + s            
    
    nxt = mod.predict(np.array(s).reshape(1, 1, 100))*18     
    nxt = [round(j-1) for j in nxt[0]]
    
    pred.append(int(nxt[0]))
    if nxt[0] == d[i]:
        corr += 1
    total += 1